In [ ]:
%%capture
!pip install --upgrade -tensorflow_hub
# !pip install -U -huggingface_hub

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../')
from eval_utils import *
sys.path.pop()

In [ ]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')
dataset_val = input('Enter the number of samples to run on (100 or 1000): ')
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')
cand_size = int(input('enter number of candidates (recommended 12 for quicker run, 50 otherwise): '))

ag_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model.to(device)
ag_pipeline = pipeline('sentiment-analysis', model=ag_model, tokenizer=ag_tokenizer)
ag_pipeline.device = next(ag_model.parameters()).device

ag_model_directory = "../../../models/bert-uncased_maskedlm_agnews_july31" #first diff
finetuned_ag_maskedlm = BertForMaskedLM.from_pretrained(ag_model_directory)
finetuned_ag_maskedlm.to(device)
ag_fill_mask = pipeline("fill-mask", model=finetuned_ag_maskedlm, tokenizer=ag_tokenizer)
ag_fill_mask.device = next(ag_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = TextFoolerJin2019

if dataset_val == '100':
    loaded_ag_100 = Dataset.load_from_disk('../data/filtered_ag_clean_100')
    ag_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_100))
    dataset = ag_100
    dataset_name = 'ag-news100'
elif dataset_val =='1000':
    loaded_ag_1000 = Dataset.load_from_disk('../data/filtered_ag_clean_1000')
    ag_1000 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_1000))
    dataset = ag_1000
    dataset_name = 'ag-news1000'
else:
    raise ValueError('Number of samples not supported')
    
if defense == "default":
    ag_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(ag_model, ag_tokenizer)
    print(ag_wrapper)
elif defense == "logit":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(ag_wrapper)


# change candidate size
attack.transformation.max_candidates = cand_size
# adjust attack threshold to match Li et al. 2023 (0.5 theshold for AgNews Universal sentences encoder):
attack.constraints[2] = UniversalSentenceEncoder(metric = 'angular', threshold = 0.5, 
                                                 window_size = 15, skip_text_shorter_than_window=True, 
                                                 compare_against_original=False)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:0
Enter the number of samples to run on (100 or 1000): 1000
Specify a defense type among "default", "logit", "maj_log", "one_hot": maj_log
enter number of candidates (recommended 12 for quicker run, 50 otherwise): 50
using num_voter = 11 and mask_pct = 0.3 with dataset = ag-news1000...
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angu

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2023-08-08 15:52:12.985385: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-08 15:52:15.171272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]
[Succeeded / Failed / Skipped / Total] 5

[Succeeded / Failed / Skipped / Total] 13 / 32 / 5 / 50:   5%|▌         | 50/1000 [3:38:17<69:07:23, 261.94s/it]

[Succeeded / Failed / Skipped / Total] 21 / 47 / 7 / 75:   8%|▊         | 75/1000 [5:22:09<66:13:11, 257.72s/it]

[Succeeded / Failed / Skipped / Total] 27 / 63 / 10 / 100:  10%|█         | 100/1000 [7:51:52<70:46:50, 283.12s/it]

[Succeeded / Failed / Skipped / Total] 35 / 79 / 11 / 125:  12%|█▎        | 125/1000 [10:01:27<70:10:14, 288.70s/it]

[Succeeded / Failed / Skipped / Total] 47 / 91 / 12 / 150:  15%|█▌        | 150/1000 [11:58:21<67:50:42, 287.34s/it]

[Succeeded / Failed / Skipped / Total] 48 / 97 / 13 / 158:  16%|█▌        | 158/1000 [12:34:51<67:02:42, 286.65s/it]